# Predicting the Newsletter Subscription Status of Minecraft Players Based on Player Characteristics and Playing Behaviors
!!! <br>
A data science project submitted in partial fulfillment of the Requirements for the DSCI 100 Introduction to Data Science Course at the University of British Columbia <br>
Jun 23, 2025

## Introduction

A research group in Computer Science at UBC, The Pacific Laboratory for Artificial Intelligence (PLAI), led by Frank Wood, is collecting data on players of the video game MineCraft in order to understand how people interact with video games and trends in their behavior. In order to target their recruitment efforts, they would like to know, **"What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?"**

In order to investigate the general question above, this project will specifically address: <br>
**Can age, total hours played, maximum duration of sessions, and frequency of sessions of a player predict of whether the player is subscribed to the game's newsletter?**

In order to explore this question, we will use players' information and actions collected from the MineCraft server set up by PLAI.

### Exploring the data

The collected data comes in two files:

**1. <i>players.csv</i>** (196 rows, 7 cols) <br>
   This file contains all unique players that can be identified by their hashedEmail. Additional characterisitics for each player are:
   - **experience** - player's experience level (Beginner, Amateur, Regular, Pro, Veteran)
   - **subscribe** - player's subscription status to the game's newsletter (TRUE for subscribed, FALSE for not subscribed)
   - **played_hours** - total hours played by the player
   - **name** - first name of the player
   - **gender** - gender of the player (Male, Female, Non-binary, Prefer not to say, Agender, Two-Spirited)
   - **Age** - age of the the player in years
     
**2. <i>sessions.csv</i>** (1535 rows, 5 cols) <br>
   This file contains all the play sessions during the period the data was collected. Each session is associated to at player by hashedEmail. It includes the  following information about each session:
   - **start_time** - start time and date of the session in the format DD/MM/YYYY HH:MM
   - **end_time** - end time and date of the session in the format DD/MM/YYYY HH:MM
   - **original_start_time** - start time of the session as tracked by the computer (number of seconds from Jan 1 1970)
   - **original_end_time** - start time of the session as tracked by the computer (number of seconds from Jan 1 1970)

In [ ]:
# Load libraries
library(tidyverse)
library(repr)

# Only display top 6 rows of data frames
options(repr.matrix.max.rows = 6)

players_data <- read_csv("data/players.csv")
sessions_data <- read_csv("data/sessions.csv")

players_data
sessions_data

### Data Wrangling

Checking for missing data, we notice that there are only two observations in the "players_data" that are missing the value for age, and only two observations in the "sessions_data" that are missing the end time. Since removing these few observations will have little impact on the large data sets, we removed them at this stage to avoid inconsistent or unexpected behavior in the downstream analyses.

Additionally, we checked whether any start_time and end_time had different dates. If all playing sessions spanned only a single date and not overnight, we could re-arrange into date, start time and end time to tidy up the data. However, there were 93 cases of sessions spanning 2 or more days, so we opted to leave the dates as they were.

In [ ]:
# Check for Missing NA values

any(is.na(players_data)) #TRUE - there are two observations without age
any(is.na(sessions_data)) #TRUE - there are two observations without end time

# Since there are only two observations wihtout end time, and two observations without age,
# we will remove them to not cause issues in downstream analyses
sessions_data_NA_omitted <- na.omit(sessions_data)
players_data_NA_omitted <- na.omit(players_data)

# check if there are any start and end dates that aren't the same for a single session
sessions_data_sep_date_time <- sessions_data |>
    separate(col = start_time, into = c("start_date", "start_time"), sep = " ") |>
    separate(col = end_time, into = c("end_date", "end_time"), sep = " ")

sessions_data_diff_start_end_date <- sessions_data_sep_date_time |>
    filter(start_date != end_date)

sessions_data_diff_start_end_date

# Yes, there are 92 cases of sessions spanning 2 (or more) days

To further tidy and prepare the data for analyses,
- in the players data: convert categorical variables to factors (fct), especially for subscribe to allow classs prediction
- in the sessions data: convert start_time and end_time from character (chr) to date type (dttm), to allow later subtraction of times for the duration calculation

The clean data frames were called "players_data_clean" and "sessions_data_clean". Taking the dimensions (dim) of the dataframes confirm that 2 observations from each were removed as expected, resulting in **194 observations** in players_data_clean and **1533 observations** in sessions_data_clean.

In [ ]:
## Convert categorical variables to factor, esp needed for subscribe, which we are interested in classifying by:
players_data_clean <- players_data_NA_omitted |>
    mutate(subscribe = as_factor(subscribe)) |>
    mutate(experience = as_factor(experience)) |>
    mutate(gender = as_factor(gender)) #|>

sessions_data_clean <- sessions_data_NA_omitted |>
    mutate(start_time = as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")) |>
    mutate(end_time = as.POSIXct(end_time, format = "%d/%m/%Y %H:%M")) |>
    select(hashedEmail:end_time) # Since we will not be using the original times

# View clean data and dimensions:
players_data_clean
dim(players_data_clean)

sessions_data_clean
dim(sessions_data_clean)

## Methods & Results

Our data analysis will comprise of the following steps
1. Explore player characteristics and behavior by visualization
2. Based on initial visualization, select variables and/or combination of variables to use in the model for predicting players' subscription status
3. Build the KNN classification model for each of the sets of variable predictors on the training set
4. Evaluate the KNN classification model for each of the sets of variable predictors on the testing set

### Exploring player characteristics and behavior by visualization
To identify potential predcitors of player subscription status, we will first explore characteristics of players (Age, Experience) and secondly players' behavior based on their playing sessions.

In [ ]:
# Loading libraries
library(tidymodels)

# set seed once for reproducibility
set.seed(2473)

First, since we are interested in predicting subscription status, it would be valuable to know what **proportion of players are subscribed**, which we find to be approximately **73.2%** (Figure 1).

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
# Proprotion of subsribed vs not substribed
ggplot(players_data_clean, aes(x = subscribe, fill = subscribe)) +
  geom_bar(position = "dodge") +
  scale_x_discrete(labels = c("FALSE" = "Not Subscribed", "TRUE" = "Subscribed")) +
  scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                    labels = c("FALSE" = "Not Subscribed", "TRUE" = "Subscribed")) +
  labs(
    x = "Subscription Status",
    y = "Number of Players",
    fill = "Subscribed", 
    title = "Figure 1. Number of Players by \n Subscription Status"
  ) +
  theme(text = element_text(size = 20),
       legend.position = "none")

players_data_clean |>
  summarise(percent_subscribed = mean(subscribe == TRUE) * 100)

Looking at **age distribution** of players and their subscription status (Figure 2), we see that there is a moderate spread in the distribution of players, with most players being between ~15 to 30 years old. There is a general tendency for younger players (under 15) to be subscribed and older players (over 30) to not be subscribed.

In [ ]:
# AGe
options(repr.plot.width=12, repr.plot.height=8)

age_dist <- players_data_clean |>
  ggplot(aes(x = Age, fill = subscribe)) +
  geom_histogram(binwidth = 3) +
  labs(x = "Age (years)", y = "Number of players", fill = "Subscription status") +
  theme(text = element_text(size = 20)) +
  ggtitle("Figure 2. Age distribution of players and subscription status") +
  scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                    labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

age_dist

Furthermore, we can visualize the **age distribution of players by experience level** (Figure 3). We see that specifcally, older players (approximately 28 years and above) at the beginner and amateur levels tend to not subscribe, while younger (under 16) Amateur, Regular and Pro players are consistently subscribed. In the middle age range, there is a mix of subcribers and non-subcribers. However, across only experience levels, there is no major difference in proportions of subscribed players (Figure 4).

In [ ]:
## !!! LOOK AT AGE DISTRIBUTION OF Each experience level

# Histogram of Age distribution by experience level
options(repr.plot.width=12, repr.plot.height=12)

player_beginner <- players_data_clean |>
  filter(experience == "Beginner")

player_amateur <- players_data_clean |>
  filter(experience == "Amateur")

player_regular <- players_data_clean |>
  filter(experience == "Regular")

player_pro <- players_data_clean |>
  filter(experience == "Pro")

player_veteran <- players_data_clean |>
  filter(experience == "Veteran")

selected_binwidth = 1

age_dist_experience <- players_data_clean |>
  ggplot(aes(x = Age, fill = subscribe)) +
  geom_histogram(binwidth = selected_binwidth) +
  facet_grid(rows = vars(experience)) +
  labs(x = "Age (years)", y = "Number of players", fill = "Subscription status") +
  theme(text = element_text(size = 20)) +
  ggtitle("Figure 3. Age distribution of players by experience level \n and subscription status") +
  scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                    labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

age_dist_experience

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

# Order from least experienced to most experienced
players_data_clean$experience <- factor(players_data_clean$experience,
                                  levels = c("Beginner", "Amateur", "Regular", "Pro", "Veteran"))

experience_plot <- players_data_clean |>
    ggplot(aes(x = experience, fill = subscribe)) +
    geom_bar(position = "fill") +
    labs(x = "Experience level",
        y = "Proportion", 
        fill = "Subscription status") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 4. Bar graph of proportions of subcribed players by experience") +
        scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

experience_plot

Briefly, looking at **proportions of subscribed players by gender** (Figure 5), we see that the proportions are quite comparable across all genders, except for the Agender group, where all players are subscribed. Individuals who identified as Two-spirited or preferred not to indicate their gender showed lower subscription rates as well, closer to 50/50. There are also some differences in distribution of gender among different experience levels (Figure 6).

In [ ]:
## Gender

options(repr.plot.width=14, repr.plot.height=6)
gender_subscribe_plot <- players_data_clean |>
    ggplot(aes(x = gender, fill = subscribe)) +
    geom_bar(position = "fill") + #fill/dodge/stack
    labs(x = "Gender",
        y = "Proportion", 
        fill = "Subscription status") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 5. Bar graph of Proportions of subscribed players by Gender") +
        scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

gender_subscribe_plot

options(repr.plot.width=12, repr.plot.height=8)

experience_gender_plot <- players_data_clean |>
    ggplot(aes(x = experience, fill = gender)) +
    geom_bar(position = "fill") +
    labs(x = "Experience level",
        y = "Proportion", 
        fill = "Gender") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 6. Bar graph of Gender proportions of players by experience") +
        # scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
        #                labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))
        scale_fill_brewer(palette = "Paired")

experience_gender_plot

Finally, from the players data, let's look at the scatter plot of **total hours played vs. age of the players** (Figure 7). We observe that that when the total time played exceeds the 10s of hours, players tend to be subscribed, regardless of the age. Consistent with previous age distribution visualizations, younger players tend to be subscribed even if they have played very few hours, while older players tend not to be subscribed. Older players also tend to have played fewer hours. Players with the most hours are between 15 and 30 years old.

In [ ]:
# Scatter plot of played hours, age?

options(repr.plot.width=12, repr.plot.height=10)

players_hours_age_plot <- ggplot(players_data_clean, aes(x = Age,
                                                   y = played_hours,
                                                   color = subscribe,
                                                   shape = subscribe)) +
    geom_point(size = 3, alpha = 0.5) +
    labs(x = "Age of player (Years)",
        y = "Total time played (Hours)", 
        color = "Subscription status",
        shape = "Subscription status") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 7. Scatter plot of total time spent playing Minecraft vs age of player") +
        scale_color_manual(values = c("FALSE" = "red", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed")) +
        scale_shape_manual(values = c("FALSE" = 16, "TRUE" = 17),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))


players_hours_age_plot

Next, from the **sessions data**, we will calculate some summary values to describe the **behavior** of players. We will look at two aspects of behavior: (mean and maximum) duration of session and the frequency of session, which we will calculate as simple the total number of sessions during the data collection period.

In [ ]:
# Calculate duration of each session
sessions_data_clean <- sessions_data_clean |>
    mutate(duration = end_time - start_time)

# Calculate mean and max duration per session and frequency (total number of sessions) for each player
sessions_data_summary <- sessions_data_clean |>
    group_by(hashedEmail) |>
    summarize(mean_duration = mean(duration),
              max_duration = max(duration),
             frequency = n())

# Use left join to merge player info and sessions summary info
# so that all players are kept, including those with no sessions
player_info_sessions_summary <- players_data_clean |>
    left_join(sessions_data_summary, by = "hashedEmail")

# For players with no sessions set mean_duration, max_duration and frequency to 0
player_info_sessions_summary <- player_info_sessions_summary |>
    mutate(mean_duration = replace_na(mean_duration, as.duration(0)),
           max_duration = replace_na(max_duration, as.duration(0)),
           frequency = replace_na(frequency, 0))

# convert all the durations to integers/doubles in order to do standardization later
player_info_sessions_summary <- player_info_sessions_summary |>
    mutate(mean_duration = as.numeric(mean_duration),
           max_duration = as.numeric(max_duration))

player_info_sessions_summary

In [ ]:
#select same binwidth for histograms
selected_binwidth = 5

# Histrogram of frequency
options(repr.plot.width=12, repr.plot.height=6)

hist_mean_freq_subscribe <- player_info_sessions_summary |>
    ggplot(aes(x = frequency, fill = subscribe)) +
    geom_histogram(binwidth = selected_binwidth) +
    labs(x = "Total number of sessions",
        y = "Number of players",
        fill = "Subscriber status") +
    theme(text = element_text(size = 20)) +
    ggtitle("Figure 8. Distribution of frequency of playing sessions") +
    scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

hist_mean_freq_subscribe


# Histogram of Mean Duration
options(repr.plot.width=12, repr.plot.height=6)

hist_mean_duration_subscribe <- player_info_sessions_summary |>
    ggplot(aes(x = mean_duration, fill = subscribe)) +
    geom_histogram(binwidth = selected_binwidth) +
    labs(x = "Mean duration per playing session of player (minutes)",
        y = "Number of players",
        fill = "Subscriber status") +
    theme(text = element_text(size = 20)) +
    ggtitle("Figure 9. Distribution of mean duration per session played") +
    scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

hist_mean_duration_subscribe

# Histogram of Max Duration
              
hist_max_duration_subscribe <- player_info_sessions_summary |>
    ggplot(aes(x = max_duration, fill = subscribe)) +
    geom_histogram(binwidth = selected_binwidth) +
    labs(x = "Maximum duration per playing session of player (minutes)",
        y = "Number of players",
        fill = "Subscriber status") +
    theme(text = element_text(size = 20)) +
    ggtitle("Figure 10. Distribution of maximum duration per session played") +
    scale_fill_manual(values = c("FALSE" = "salmon", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))

hist_max_duration_subscribe




In [ ]:
# Scatter plot of Age vs max duration

options(repr.plot.width=12, repr.plot.height=10)

players_hours_age_plot <- ggplot(player_info_sessions_summary, aes(x = Age,
                                                   y = max_duration,
                                                   color = subscribe,
                                                   shape = subscribe)) +
    geom_point(size = 3, alpha = 0.5) +
    labs(x = "Age of player (Years)",
        y = "Max duration of playing session (Minutes)", 
        color = "Subscription status",
        shape = "Subscription status") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 11. Scatter plot of Max duration of playing session vs age of player") +
        scale_color_manual(values = c("FALSE" = "red", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed")) +
        scale_shape_manual(values = c("FALSE" = 16, "TRUE" = 17),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))


players_hours_age_plot

# Scatter plot of Age vs frequency

players_hours_age_plot <- ggplot(player_info_sessions_summary, aes(x = Age,
                                                   y = frequency,
                                                   color = subscribe,
                                                   shape = subscribe)) +
    geom_point(size = 3, alpha = 0.5) +
    labs(x = "Age of player (Years)",
        y = "Frequency of sessions (count)", 
        color = "Subscription status",
        shape = "Subscription status") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 12. Scatter plot of frequency of playing session vs age of player") +
        scale_color_manual(values = c("FALSE" = "red", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed")) +
        scale_shape_manual(values = c("FALSE" = 16, "TRUE" = 17),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))


players_hours_age_plot


# Scatter plot of frequency vs max duration
players_hours_age_plot <- ggplot(player_info_sessions_summary, aes(x = frequency,
                                                   y = max_duration,
                                                   color = subscribe,
                                                   shape = subscribe)) +
    geom_point(size = 3, alpha = 0.5) +
    labs(x = "Frequency of sessions (count)",
        y = "Max duration (minutes)", 
        color = "Subscription status",
        shape = "Subscription status") +
        theme(text = element_text(size = 20)) +
        ggtitle("Figure 13. Scatter plot of frequency of playing session vs age of player") +
        scale_color_manual(values = c("FALSE" = "red", "TRUE" = "steelblue"),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed")) +
        scale_shape_manual(values = c("FALSE" = 16, "TRUE" = 17),
                       labels = c("FALSE" = "Not subscribed", "TRUE" = "Subscribed"))


players_hours_age_plot

### Building the KNN Classification Models on the Training Data
Based on the above histograms and scatterplots, there there seems to be a weaker relationship between subscription status and mean duration of sessions of players. Therefore, we will omit it from out predictors.

Let's try to create a classification model with the following numeric predictors:
-  age
-  total time spent playing the game
-  max session duration
-  frequency of sessions
   
Specifically, we will look at the following pairs of predictors and all 4 predictors that appear to affect subscription:

1. Age + total hours played
2. Age + Max duration of a session
3. Age + Frequency
4. All 4 predictors above: Age + total hours played + Max duration of a session + Frequency

In [ ]:
# For readability, we will call the combined player information and sessions summary player_data as just data
player_data <- player_info_sessions_summary

# Split data into training and testing subsets

player_split <- initial_split(player_data, prop = 0.75, strata = subscribe)
player_train <- training(player_split)
player_test <- testing(player_split)

# 10-fold cross-validation on the training set
player_vfold <- vfold_cv(player_train, v = 10, strata = subscribe)

# Create the knn model specifications for tuning K
player_knn <- nearest_neighbor(weight_func = "rectangular",
                               neighbors = tune()) |>
                set_engine("kknn") |>
                set_mode("classification")

# Create a tibble of K values to try
k_vals <- tibble(neighbors = seq(from = 5, to = 50, by = 2))

# The above code will be used for all the models

In [ ]:
# === Predicting Subscription status by Age + Total Hours played ===

recipe_age_hours <- recipe(subscribe ~ Age + played_hours,
                     data = player_train) |>
                step_center(all_predictors()) |>
                step_scale(all_predictors())

# Use tune_grid to fit the model
# Collect the metricsm filter for "accuracy"
# Plot accuracy vs K
knn_results_age_hours <- workflow() |>
    add_recipe(recipe_age_hours) |>
    add_model(player_knn) |>
    tune_grid(resamples = player_vfold, grid = k_vals) |>
    collect_metrics()

knn_results_age_hours

# Tune k-NN model with 5-fold CV
knn_accuracy_age_hours <- knn_results_age_hours |>
    filter(.metric == "accuracy")

knn_accuracy_age_hours

# Plot accuracy vs. number of neighbors
knn_age_hours_plot <- knn_accuracy_age_hours |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    ggtitle("Figure 14. Accuracy vs. Number of Neighbors (K) of KNN Classification Model using Age and Total Hours Played as predictors") +
    theme(text = element_text(size = 12))

knn_age_hours_plot

knn_results_age_hours_sorted <- knn_accuracy_age_hours |>
    arrange(desc(mean))

knn_results_age_hours_sorted

best_k_age_hours <- knn_results_age_hours_sorted |>
    head(1) |>
    pull(neighbors)

best_k_age_hours

In [ ]:
# === Predicting Subscription status by Age + Maximum duration of a session ===

# Create recipes for pairs of predictors using training data
recipe_age_max <- recipe(subscribe ~ Age + max_duration,
                     data = player_train) |>
                step_center(all_predictors()) |>
                step_scale(all_predictors())

# Use tune_grid to fit the model
# Collect the metricsm filter for "accuracy"
# Plot accuracy vs K

knn_results_age_max <- workflow() |>
    add_recipe(recipe_age_max) |>
    add_model(player_knn) |>
    tune_grid(resamples = player_vfold, grid = k_vals) |>
    collect_metrics()

knn_results_age_max

# Tune k-NN model with 5-fold CV
knn_accuracy_age_max <- knn_results_age_max |>
    filter(.metric == "accuracy")

knn_accuracy_age_max

# Plot accuracy vs. number of neighbors
knn_age_max_plot <- knn_accuracy_age_max |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    ggtitle("Figure 15. Accuracy vs. Number of Neighbors (K) of KNN Classification Model using Age and Maximum Session Duration as predictors") +
    theme(text = element_text(size = 12))

knn_age_max_plot

knn_results_age_max_sorted <- knn_accuracy_age_max |>
    arrange(desc(mean))

knn_results_age_max_sorted

best_k_age_max <- knn_results_age_max_sorted |>
    head(1) |>
    pull(neighbors)

best_k_age_max

In [ ]:
# === Predicting Subscription status by Age + Frequency of Sessions ===

# Create recipes for pairs of predictors using training data
recipe_age_freq <- recipe(subscribe ~ Age + frequency,
                     data = player_train) |>
                step_center(all_predictors()) |>
                step_scale(all_predictors())

# Use tune_grid to fit the model
# Collect the metricsm filter for "accuracy"
# Plot accuracy vs K

knn_results_age_freq <- workflow() |>
    add_recipe(recipe_age_freq) |>
    add_model(player_knn) |>
    tune_grid(resamples = player_vfold, grid = k_vals) |>
    collect_metrics()

knn_results_age_freq

knn_accuracy_age_freq <- knn_results_age_freq |>
    filter(.metric == "accuracy")

knn_accuracy_age_max

# Plot accuracy vs. number of neighbors
knn_age_freq_plot <- knn_accuracy_age_freq |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    ggtitle("Figure 16. Accuracy vs. Number of Neighbors (K) of KNN Classification Model using Age and Frequency of Sessions as predictors") +
    theme(text = element_text(size = 12))

knn_age_freq_plot

knn_results_age_freq_sorted <- knn_accuracy_age_freq |>
    arrange(desc(mean))

knn_results_age_freq_sorted

best_k_age_freq <- knn_results_age_freq_sorted |>
    head(1) |>
    pull(neighbors)

best_k_age_freq

In [ ]:
# === Predicting Subscription status by Age + Total Hours played + Max duration of a session + Frequency of Sessions ===

# Create recipes for pairs of predictors using training data
recipe_all <- recipe(subscribe ~ Age + played_hours + max_duration + frequency,
                     data = player_train) |>
                step_center(all_predictors()) |>
                step_scale(all_predictors())

# Use tune_grid to fit the model
# Collect the metricsm filter for "accuracy"
# Plot accuracy vs K

knn_results_all <- workflow() |>
    add_recipe(recipe_all) |>
    add_model(player_knn) |>
    tune_grid(resamples = player_vfold, grid = k_vals) |>
    collect_metrics()

knn_results_all

# Tune k-NN model with 5-fold CV
knn_accuracy_all <- knn_results_all |>
    filter(.metric == "accuracy")

knn_accuracy_all

# Plot accuracy vs. number of neighbors
knn_all_plot <- knn_accuracy_all |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    ggtitle("Figure 17. Accuracy vs. Number of Neighbors (K) of KNN Classification Model using \n Age, Total Hours Played, Maximum session duration and frequnecy of sessions as predictors") +
    theme(text = element_text(size = 12))

knn_all_plot

knn_results_all_sorted <- knn_accuracy_all |>
    arrange(desc(mean))

knn_results_all_sorted

best_k_all <- knn_results_all_sorted |>
    head(1) |>
    pull(neighbors)

best_k_all

Of the 4 models above using different predictor combinations, it appears that the last three (listed below) have a slight ability to predict subcription, as there are K values with accuracy estimates that peak above the plateau accuracy level:

Age + Max duration of a session
Age + Frequency
All 4 predictors above: Age + total hours played + Max duration of a session + Frequency

### Predicting and Evalutating the Subscription Status of Players on the Testing Data

Finally, we will use the models and selected best Ks for each set of predictors on the testing data and evaluate each's performance by calculating accuracy, precision, recall metrics and viewing the confusion matrix.

In [ ]:
# === Predicting Subscription status by Age + Total Hours played ===

# Re-train model with selected K
knn_spec_age_hours <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_age_hours) |>
    set_engine("kknn") |>
    set_mode("classification")

knn_fit_age_hours <- workflow() |>
    add_recipe(recipe_age_hours) |>
    add_model(knn_spec_age_hours) |>
    fit(data = player_train)

# Predict on test set
player_pred_age_hours <- predict(knn_fit_age_hours, player_test) |>
    bind_cols(player_test)

# Check order of labels in the subscribe variable - [1] False [2] True
player_pred_age_hours |> pull(subscribe) |> levels()

player_pred_age_hours |>
    metrics(truth = subscribe, estimate = .pred_class) |>
    filter(.metric == "accuracy")

# Precision
player_pred_age_hours |>
    precision(truth = subscribe, estimate = .pred_class, event_level = "second")

# Recall
player_pred_age_hours |>
    recall(truth = subscribe, estimate = .pred_class, event_level = "second")

# Confusion matrix
player_pred_age_hours |>
    conf_mat(truth = subscribe, estimate = .pred_class)

In [ ]:
# === Predicting & Evaluating Subscription status by Age + Max Session Duration ===

# Re-train model with best K for this predictor combo
knn_spec_age_max <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_age_max) |>
  set_engine("kknn") |>
  set_mode("classification")

# Fit the model using the recipe with Age + max_duration
knn_fit_age_max <- workflow() |>
  add_recipe(recipe_age_max) |>
  add_model(knn_spec_age_max) |>
  fit(data = player_train)

# Predict on the test set
player_pred_age_max <- predict(knn_fit_age_max, player_test) |>
  bind_cols(player_test)

# Optional: check label levels
player_pred_age_max |> pull(subscribe) |> levels()

# Accuracy
player_pred_age_max |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

# Precision (for positive class: subscribed == TRUE)
player_pred_age_max |>
  precision(truth = subscribe, estimate = .pred_class, event_level = "second")

# Recall (for positive class: subscribed == TRUE)
player_pred_age_max |>
  recall(truth = subscribe, estimate = .pred_class, event_level = "second")

# Confusion matrix
player_pred_age_max |>
  conf_mat(truth = subscribe, estimate = .pred_class)

In [ ]:
# === Predicting & Evaluating Subscription status by Age + Frequency of Sessions ===

# Define model spec with best K
knn_spec_age_freq <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_age_freq) |>
  set_engine("kknn") |>
  set_mode("classification")

# Fit the model using the Age + frequency recipe
knn_fit_age_freq <- workflow() |>
  add_recipe(recipe_age_freq) |>
  add_model(knn_spec_age_freq) |>
  fit(data = player_train)

# Predict on the test set
player_pred_age_freq <- predict(knn_fit_age_freq, player_test) |>
  bind_cols(player_test)

# Optional: check label levels
player_pred_age_freq |> pull(subscribe) |> levels()

# Accuracy
player_pred_age_freq |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

# Precision (for subscribed == TRUE)
player_pred_age_freq |>
  precision(truth = subscribe, estimate = .pred_class, event_level = "second")

# Recall (for subscribed == TRUE)
player_pred_age_freq |>
  recall(truth = subscribe, estimate = .pred_class, event_level = "second")

# Confusion matrix
player_pred_age_freq |>
  conf_mat(truth = subscribe, estimate = .pred_class)

In [ ]:
# === Predicting & Evaluating Subscription Status with All Predictors ===
# (Age + Total Hours Played + Max Session Duration + Frequency of Sessions)

# Define k-NN model with best K
knn_spec_all <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_all) |>
  set_engine("kknn") |>
  set_mode("classification")

# Fit the model with full predictor set
knn_fit_all <- workflow() |>
  add_recipe(recipe_all) |>
  add_model(knn_spec_all) |>
  fit(data = player_train)

# Predict on the test set
player_pred_all <- predict(knn_fit_all, player_test) |>
  bind_cols(player_test)

# Optional: check label levels
player_pred_all |> pull(subscribe) |> levels()

# Accuracy
player_pred_all |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

# Precision (for subscribed == TRUE)
player_pred_all |>
  precision(truth = subscribe, estimate = .pred_class, event_level = "second")

# Recall (for subscribed == TRUE)
player_pred_all |>
  recall(truth = subscribe, estimate = .pred_class, event_level = "second")

# Confusion matrix
player_pred_all |>
  conf_mat(truth = subscribe, estimate = .pred_class)

In summary, all models had a moderate, similar accuracy and precision (mid 70%) , but a high recall (close to 100%), meaning they tended to predict people as subcribers even when they were not.

**Table summary of KNN models with various predictors and their performance metrics (accuracy, precision, recall)**
                                                         
| Model|Age | Total Hours Played | Max Duration of Session | Frequency of Sessions | Accuracy | Precision | Recall |
|------|----|--------------------|-------------------------|-----------------------|----------|-----------|--------|
1      | x  | x                  |                         |                       |  0.755   |  0.761    |  1     |
2      | x  |                    |   x                     |                       |  0.735   |  0.745    | 0.972  |
3      | x  |                    |                         | x                     |  0.755   |  0.761    | 0.972  |
4      | x  | x                  |    x                    |    x                  | 0.735    |   0.735   |   1    |

The KNN Predictor Model using predictors Age and Total hours played (Model 1) and the Model using age and Frequency of sessions (Model 3) were the most successful. However, even the accuracy results for these were only marginally (2%) better than a theoretical predictor that predicts all of players to be subscribers. Recall from Figure 1 that around 73.2% of players are subscribers. Models using age and max duration of session (Model 2) and using all 4 predictors (Model 4) gave an accuracy that was essentially equal to the proportion of players that are subscribers, and do not appear to have any meaningful predictive power.

## Discussion

Overall, the KNN Classification model was slightly effective in predicting player subscription status when using either age and total hours played as predictors or age (model 1) and frequency of sessions (model 3).

The results are not surprising, considering that based on the early visualizations of the data, the only patterns that could be observed were mainly at the extreme ends of the data where there were few observations (young age, old age, a lot of player activity, etc.), whereas where the players clustered in the middle age group or with low to moderate playing activity, there was a mix of subscribers and non-subscibers with no apparent trend from the variables we looked at.

Impact

It would be of interest to combine Age, Total Hours played and Maximum Frequency of sessions as predictors for the KNN model to see if they improve the performance at all.

In this analysis using KNN classification, we were restricted to numerical variables. As there appeared to be some differences in subcription when looking at gender and when looking at age distribution and experience together, these may be useful to factor in when building a model. Using more advanced tools for classification that allow use of categorical variables as predictors, such as decision trees and support vector machines, two very popular but more advanced classification methods (Campbell et al., 2024).


## References

Campbell, T., Timbers, T., & Lee, M. (2024, August 21). Data science. https://datasciencebook.ca/ 